In [1]:
import fastf1
import pandas as pd 
import numpy as np
import re
from matplotlib import pyplot as plt
print(fastf1.__version__)
from fastf1 import utils

2.3.3


In [2]:
#BIen
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Monaco"
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
session.load()
laps = session.load_laps(with_telemetry=True)
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()
laps["PitInSeconds"]=laps['PitInTime'].dt.total_seconds()
laps["PitOutSeconds"]=laps['PitOutTime'].dt.total_seconds()
laps["Sector1Time"]=laps['Sector1Time'].dt.total_seconds()
laps["Sector2Time"]=laps['Sector2Time'].dt.total_seconds()
laps["Sector3Time"]=laps['Sector3Time'].dt.total_seconds()
racelaps=laps[["Driver","LapNumber","LapTimeSeconds","Compound"]]
#racelaps.to_csv('Monaco2023.csv')

core           INFO 	Loading data for Monaco Grand Prix - Race [v2.3.3]
api            INFO 	Using cached data for driver_info
core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '27', '20', '2', '77', '18', '23', '21', '81', '4', '22', '63', '10', '16', '44', '55', '31', '14', '24', '11']
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1220: FutureWarning: `Sess

In [40]:
# Group the racelaps by compound
grouped_laps = racelaps.groupby("LapNumber",'Compound')

# Filter the groups by desired compounds
medium_laps = grouped_laps.get_group('MEDIUM')
hard_laps = grouped_laps.get_group('HARD')

# Calculate the mean lap time for each lap of the medium compound
medium_lap_time = medium_laps.groupby('LapNumber')['LapTimeSeconds'].mean().reset_index()
medium_lap_time = medium_lap_time.rename(columns={'LapNumber': 'Lap', 'LapTimeSeconds': 'MediumAvgLapTime'})

# Calculate the mean lap time for each lap of the hard compound
hard_lap_time = hard_laps.groupby('LapNumber')['LapTimeSeconds'].mean().reset_index()
hard_lap_time = hard_lap_time.rename(columns={'LapNumber': 'Lap', 'LapTimeSeconds': 'HardAvgLapTime'})

# Merge the two DataFrames on the 'Lap' column using an outer join
lap_times_df = pd.merge(medium_lap_time, hard_lap_time, on='Lap', how='outer')

# Fill in missing values with NaN
lap_times_df = lap_times_df.fillna(value=np.nan)

# Convert the DataFrame to long format
lap_times_df = lap_times_df.melt(id_vars=['Lap'], var_name='Compound', value_name='AvgLapTime')

# Print the resulting DataFrame
print(lap_times_df)

lap_times_df.to_csv("MIAMIcompounds2023.csv")





     Lap          Compound  AvgLapTime
0      1  MediumAvgLapTime         NaN
1      2  MediumAvgLapTime   95.224818
2      3  MediumAvgLapTime   93.972800
3      4  MediumAvgLapTime   93.908600
4      5  MediumAvgLapTime   93.642300
..   ...               ...         ...
109   27    HardAvgLapTime   92.933300
110   28    HardAvgLapTime   92.905500
111   29    HardAvgLapTime   92.815750
112   30    HardAvgLapTime   92.893900
113   31    HardAvgLapTime   92.946850

[114 rows x 3 columns]


In [77]:
rainylaps=laps.loc[laps["LapNumber"] >= 30]
laps=rainylaps.loc[laps["LapNumber"] <= 35]
VER_laps=laps.loc[racelaps['Driver'] == 'OCO']
PER_laps =laps.loc[racelaps['Driver'] == 'HAM']
SAI_laps=laps.loc[racelaps['Driver'] == 'SAI']
combined_laps = pd.concat([VER_laps,SAI_laps,PER_laps])
combined_laps.to_csv("racepacemonacoSAI.csv")

In [29]:
# Group the racelaps by compound and lap number
grouped_laps = racelaps.groupby(['Compound', 'LapNumber'])

# Calculate the mean lap time for each group
mean_lap_time = grouped_laps['LapTimeSeconds'].mean()

# Filter the resulting Series to only include the 'M' compound
medium_lap_time = mean_lap_time.loc['MEDIUM']

# Create a new DataFrame from the filtered Series
medium_lap_time_df = pd.DataFrame({'MediumAvgLapTime': medium_lap_time})

# Print the resulting DataFrame
print(medium_lap_time_df)




# Calculate the mean lap time for each group

# Filter the resulting Series to only include the 'M' compound
hard_lap_time = mean_lap_time.loc['HARD']

# Create a new DataFrame from the filtered Series
hard_lap_time_df = pd.DataFrame({'HARDAvgLapTime': medium_lap_time})

# Print the resulting DataFrame
#print(hard_lap_time_df)



           MediumAvgLapTime
LapNumber                  
1                       NaN
2                 95.224818
3                 93.972800
4                 93.908600
5                 93.642300
6                 93.482100
7                 93.310500
8                 93.435200
9                 93.496900
10                93.568300
11                93.151400
12                93.014700
13                93.281800
14                93.952300
15                95.211556
16                93.157833
17                94.589167
18                94.127750
19                93.221000
20                94.571667
21                93.375000
22                95.786000
23                92.266000
24                97.282000
32               107.897000
33                92.264000
34                99.918000
35                91.599000
36                91.403000
37                97.053667
38                95.326000
39                92.025000
40                94.810000
41                91

In [3]:
#Results
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Monaco"
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
session.load()
results = session.results
print(results)
#results.to_csv('ResultsMonacoR2023.csv')

core           INFO 	Loading data for Monaco Grand Prix - Race [v2.3.3]
api            INFO 	Using cached data for driver_info
core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '27', '20', '2', '77', '18', '23', '21', '81', '4', '22', '63', '10', '16', '44', '55', '31', '14', '24', '11']


   DriverNumber BroadcastName Abbreviation         TeamName TeamColor  \
1             1  M VERSTAPPEN          VER  Red Bull Racing    3671C6   
27           27  N HULKENBERG          HUL     Haas F1 Team    B6BABD   
20           20   K MAGNUSSEN          MAG     Haas F1 Team    B6BABD   
2             2    L SARGEANT          SAR         Williams    37BEDD   
77           77      V BOTTAS          BOT       Alfa Romeo    C92D4B   
18           18      L STROLL          STR     Aston Martin    358C75   
23           23       A ALBON          ALB         Williams    37BEDD   
21           21    N DE VRIES          DEV       AlphaTauri    5E8FAA   
81           81     O PIASTRI          PIA          McLaren    F58020   
4             4      L NORRIS          NOR          McLaren    F58020   
22           22     Y TSUNODA          TSU       AlphaTauri    5E8FAA   
63           63     G RUSSELL          RUS         Mercedes    6CD3BF   
10           10       P GASLY          GAS         

In [79]:
#Info driver 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Monaco"
ses = 'Q'
session = fastf1.get_session(year, wknd, ses)
weekend = session.event
driver='VER'
session.load()
lap = session.laps.pick_driver(driver).pick_fastest()
lap.to_csv('VERQMon.csv')

#Telemetry
x = lap.telemetry['X']              
y = lap.telemetry['Y']
speed = lap.telemetry['Speed'] 
tel=pd.DataFrame({'X': x, 'Y': y,'Speed': speed })
#tel.to_csv('telALOQMonaco.csv')



core           INFO 	Loading data for Monaco Grand Prix - Qualifying [v2.3.3]
api            INFO 	Using cached data for driver_info
core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '23', '2', '24', '22', '14', '27', '20', '63', '77', '31', '44', '16', '10', '4', '55', '18', '11', '81', '21']


In [ ]:
#Info driver 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 2
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
weekend = session.event
driver='PIA'
session.load()
laps= session.laps.pick_driver(driver)
print(type(laps))
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()
#laps.to_csv('PIASAUDIR.csv')

In [ ]:
#pickSpecificlap
driver = 'NOR'
laps = session.laps.pick_driver(driver)
lapLando=laps[laps['LapNumber']==6].iloc[0]
lapLando

In [ ]:
#Telemetry 
x = lapLando.telemetry['X']              
y = lapLando.telemetry['Y']
color = lap.telemetry['Speed'] 
telLogan=pd.DataFrame({'X': x, 'Y': y,'Speed': color})
telLogan.to_csv('telLandoQ1.csv')

In [ ]:
#Info driver 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2021
wknd = "Sochi"
ses = 'Q'
session = fastf1.get_session(year, wknd, ses)
weekend = session.event
driver='SAI'
session.load()
laplando = session.laps.pick_driver(driver).pick_fastest()
lap.to_csv('CarlosRUSSIA2021.csv')


In [ ]:
x = laplando.telemetry['X']              
y = laplando.telemetry['Y']
color = lap.telemetry['Speed'] 
telLando=pd.DataFrame({'X': x, 'Y': y,'Speed': color})
telLando.to_csv('telCatlosRUSSIA2021.csv')

In [ ]:
#Race Pace

fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Australia"
ses = 'R'
race = fastf1.get_session(year, wknd, ses)

# Extract data for each lap
laps = race.load_laps()

# Convert laptimes to seconds
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()

# To get accurate laps only, we exclude in/out laps
laps = laps.loc[(laps['PitOutTime'].isnull() & laps['PitInTime'].isnull())]
laps.to_csv('RacePaceAustralia2023.csv')

In [ ]:
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 1
ses = 'R'
race = fastf1.get_session(year, wknd, ses)

# Extract data for each lap
laps = race.load_laps()

# Convert laptimes to seconds
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()

# To get accurate laps only, we exclude in/out laps
laps = laps.loc[(laps['PitOutTime'].isnull() & laps['PitInTime'].isnull())]
laps=laps.loc[laps["Team"]=="McLaren"]
laps.to_csv('RacePaceMclarenBAH2023.csv')

In [ ]:
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Australia"
ses = 'R'
race = fastf1.get_session(year, wknd, ses)
laps = race.load_laps()
drivers=laps['Driver']

In [ ]:
year, grand_prix, session = 2022, 'Saudi Arabia', 'Q'
quali = fastf1.get_session(year, grand_prix, session)
quali.load() 

In [ ]:
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 3
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
session.load()
driver_1, driver_2 = 'NOR', 'LEC'
laps_driver_1 = session.laps.pick_driver(driver_1)
laps_driver_2 = session.laps.pick_driver(driver_2)

# Select the fastest lap
fastest_driver_1 = laps_driver_1.pick_fastest()
fastest_driver_2 = laps_driver_2.pick_fastest()

# Retrieve the telemetry and add the distance column
telemetry_driver_1 = fastest_driver_1.get_telemetry().add_distance()
telemetry_driver_2 = fastest_driver_2.get_telemetry().add_distance()
team_driver_1 = fastest_driver_1['Team']
team_driver_2 = fastest_driver_2['Team']

delta_time, ref_tel, compare_tel = utils.delta_time(fastest_driver_1, fastest_driver_2)
telemetry_driver_1.to_csv("DRS.csv")

In [18]:
#FASTEST LAPS 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 5
ses = 'Q'
session = fastf1.get_session(year, wknd, ses)
drivers=["PER","VER","LEC","SAI","HAM","RUS","STR","ALO","NOR","PIA",
         "DEV","TSU","OCO","GAS","ZHO","BOT","MAG","HUL","ALB","SAR"]
session.load()
for driver in drivers: 
    fastest_lap = session.laps.pick_driver(driver).pick_fastest()
    tel=fastest_lap.get_telemetry().add_distance()
    fastest_lap.to_csv(f"fastestlap{driver}Miami2023.csv")

core           INFO 	Loading data for Miami Grand Prix - Qualifying [v2.3.3]
api            INFO 	Using cached data for driver_info
core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1411: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last).reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1411: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last)

In [3]:
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 4
ses = 'SS'
session = fastf1.get_session(year, wknd, ses)
drivers=["PER","VER","LEC","SAI","HAM","RUS","STR","ALO","NOR","PIA",
         "DEV","TSU","OCO","GAS","ZHO","BOT","MAG","HUL","ALB","SAR"]
session.load()
for driver in drivers: 
    lapsdriver = session.laps.pick_driver(driver)
    medium=lapsdriver.loc[lapsdriver["Compound"]=="MEDIUM"]
    fastest_lap=medium.pick_fastest()
    fastest_lap.to_csv(f"SS{driver}BAKU2023.csv")

core           INFO 	Loading data for Azerbaijan Grand Prix - Sprint Shootout [v2.3.3]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1411: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last).reset_index(drop=True)
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1411: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last).reset_index(drop=True)
api            INFO 	Using cached data for car_data
api        

In [ ]:
year = 2023
wknd = 4

event = fastf1.get_event(2023, 4)
ss=event.get_session_name(3)
ss.load()

In [ ]:
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 3
ses = 'Q'
driver="NOR"
session = fastf1.get_session(year, wknd, ses)
session.load()
fastest_lap = session.laps.pick_driver(driver).pick_fastest()
tel=fastest_lap.get_telemetry().add_distance()
tel.to_csv("DRSNOR2.csv")

In [9]:
#MONACO STRATEGY 2021
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = 4
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
session.load()
laps = session.load_laps(with_telemetry=True)
laps['LapTimeSeconds'] = laps['LapTime'].dt.total_seconds()
laps["PitInSeconds"]=laps['PitInTime'].dt.total_seconds()
laps["PitOutSeconds"]=laps['PitOutTime'].dt.total_seconds()
laps["Sector1Time"]=laps['Sector1Time'].dt.total_seconds()
laps["Sector2Time"]=laps['Sector2Time'].dt.total_seconds()
laps["Sector3Time"]=laps['Sector3Time'].dt.total_seconds()
#laps.to_csv('telMonacoR2021.csv')

core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v2.3.3]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '16', '14', '55', '44', '18', '63', '4', '22', '81', '23', '20', '10', '31', '2', '27', '77', '24', '21']
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1220: FutureWarning: `Session.load_laps` is deprecated and will beremoved in a future version.
Use `Session.load` inste

In [10]:
driver_stints = laps[['Driver', 'Stint', 'Compound', 'LapNumber']].groupby(
    ['Driver', 'Stint', 'Compound']
).count().reset_index()
driver_stints = driver_stints.rename(columns={'LapNumber': 'StintLength'})

driver_stints = driver_stints.sort_values(by=['Stint'])

In [11]:
driver_stints.to_csv("StintsBAKU2023.csv")

In [60]:
#Box or not to box 
fastf1.Cache.enable_cache("/Users/ainefernandez/documents/itam/8SEMESTRE/F1 Python")
year = 2023
wknd = "Monaco"
ses = 'R'
session = fastf1.get_session(year, wknd, ses)
session.load()
laps = session.load_laps(with_telemetry=True)
laps['RaceLapNumber'] = laps['LapNumber'] - 1
# Starting from lap 45 it started raining
laps = laps.loc[laps['RaceLapNumber'] >= 51]


core           INFO 	Loading data for Monaco Grand Prix - Race [v2.3.3]
api            INFO 	Using cached data for driver_info
core        WARNING 	Failed to load data from Ergast API! (This is expected for recent sessions)
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '27', '20', '2', '77', '18', '23', '21', '81', '4', '22', '63', '10', '16', '44', '55', '31', '14', '24', '11']
/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1220: FutureWarning: `Sess

In [61]:
drivers = pd.unique(laps['Driver'])

telemetry = pd.DataFrame()

# Telemetry can only be retrieved driver-by-driver
for driver in drivers:
    driver_laps = laps.pick_driver(driver)
    
    # Since we want to compare distances, we need to collect telemetry lap-by-lap to reset the distance
    for lap in driver_laps.iterlaps():
        driver_telemetry = lap[1].get_telemetry().add_distance()
        driver_telemetry['Driver'] = driver
        driver_telemetry['Lap'] = lap[1]['RaceLapNumber']
        driver_telemetry['Compound'] = lap[1]['Compound']
    
        telemetry = telemetry.append(driver_telemetry)
        

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

...

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  telemetry = telemetry.append(driver_telemetry)
/var/folders/z3/f6b2q8h90wsgpsmjqkk2lnt80000gn/T/ipykernel_91211/3742093839.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [62]:
telemetry = telemetry[['Lap', 'Distance', 'Compound', 'Speed', 'X','Y']]

# Everything that's not intermediate will be "slick"
telemetry['Compound'].loc[telemetry['Compound'] != 'INTERMEDIATE'] = 'SLICK'

num_minisectors = 25

# What is the total distance of a lap?
total_distance = max(telemetry['Distance'])

# Generate equally sized mini-sectors 
minisector_length = total_distance / num_minisectors

minisectors = [0]

for i in range(0, (num_minisectors - 1)):
    minisectors.append(minisector_length * (i + 1))

telemetry['Minisector'] =  telemetry['Distance'].apply(
  lambda z: (
    minisectors.index(
      min(minisectors, key=lambda x: abs(x-z)))+1
  )
)
average_speed = telemetry.groupby(['Lap', 'Minisector', 'Compound'])['Speed'].mean().reset_index()
fastest_compounds = average_speed.loc[average_speed.groupby(['Lap', 'Minisector'])['Speed'].idxmax()]

# Get rid of the speed column and rename the Compound column
fastest_compounds = fastest_compounds[['Lap', 'Minisector', 'Compound']].rename(columns={'Compound': 'Fastest_compound'})
telemetry = telemetry.merge(fastest_compounds, on=['Lap', 'Minisector'])

# Order the data by distance to make matploblib does not get confused
telemetry = telemetry.sort_values(by=['Distance'])

# Assign integer value to the compound because that's what matplotlib wants
telemetry.loc[telemetry['Fastest_compound'] == "INTERMEDIATE", 'Fastest_compound_int'] = 1
telemetry.loc[telemetry['Fastest_compound'] == "SLICK", 'Fastest_compound_int'] = 2


In [63]:
telemetry.to_csv("Monaco2023.csv")
